# Generate Labeled Training Images from NAIP and GM-SEUS

In [16]:
# Import necessary libraries
import os
import random
import rasterio
import geopandas as gpd
import numpy as np
import re
import shutil
import glob
from collections import Counter
from shapely.geometry import Point, Polygon, MultiPolygon, box
from rasterio.merge import merge
from rasterio.windows import Window
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Load config file
def load_config(filename):
    config = {}
    with open(filename, 'r') as f:
        for line in f:
            # Strip whitespace and split by '='
            key, value = line.strip().split('=')
            # Try to convert to numeric values if possible
            try:
                value = float(value) if '.' in value else int(value)
            except ValueError:
                pass  # Leave as string if not a number
            config[key] = value
    return config

## Set Paths and Variables

In [17]:
# Set folder paths
wd = r'S:\Users\stidjaco\R_files\BigPanel'
downloaded_path = os.path.join(wd, r'Data\Downloaded')
derived_path = os.path.join(wd, r'Data\Derived')
derivedTemp_path = os.path.join(derived_path, r'intermediateProducts')
figure_path = os.path.join(wd, r'Figures')

# Set a final gmseus arrays path and panels path
gmseusArraysFinalPath = os.path.join(derived_path, r'GMSEUS/GMSEUS_Arrays_Final.shp')
gmseusPanelsFinalPath = os.path.join(derived_path, r'GMSEUS/GMSEUS_Panels_Final.shp')

# # Whole labeled images path
# labelImgsWholePath = os.path.join(derived_path, r'labelImgsWhole')
# # Set out paths for images and masks
# imagesPath = os.path.join(derived_path, r'GMSEUS/LabeledImages/images') # inputs
# masksPath = os.path.join(derived_path, r'GMSEUS/LabeledImages/masks') # targets

# Whole labeled images path (custom non cloud drive -- local) and duplicate images path from GEE export tiling
labelImgsWholePath = os.path.join(r'F:\Data\BigPanel\labelImgsWhole')
duplicateOrigPath = os.path.join(r'F:\Data\BigPanel\labelImgsWhole\duplicateOrig')

# Set out paths for images and masks
imagesPath = os.path.join(r'F:\Data\BigPanel\output\images') # inputs
masksPath = os.path.join(r'F:\Data\BigPanel\output\masks') # targets

# Load the config from the text file
config = load_config(os.path.join(wd, 'Code/config.txt'))

# Load variables from the config file
overlapDist = config['overlapDist'] # 190 meters, Set a overlap distance for checking if points/mismatched geometries between Solar PV datasets are duplicates
minPanelRowArea = config['minPanelRowArea'] # 15 m2, minimum area for a single panel row from the 1st percentile panel area from Stid et al., 2022
maxPanelRowArea = config['maxPanelRowArea'] # 254 m2 95th perccentile for a single panel row from Stid et al., 2022. MSU Solar Carport has max 1890m2
minNumPanelRows = config['minNumPanelRows'] # 3 panels, minimum number of panels rows to form a ground mounted solar array, definition from Stid et al., 2022
minPmArRatio = config['minPmArRatio'] # 18.8%, 20% was minimum ratio of panel perimeter to area ratio for panels from Stid et al., 2022, MSU Solar Carport has min 18.9%
panelArrayBuff = config['panelArrayBuff'] # 10m buffer, 20m maximum distance between panel rows to form an array. We used 5m in Stid et al., 2022, but there are lower packing factors at greater latitudes (nativeID: '1229957948')
arrayArrayBuff = config['arrayArrayBuff'] # 20m buffer, 40m maximum distance between arrays subsections of the same mount type to form a complete array. In Stid et al., 2022, we used 50m, but we checked for same installation year in addition to mount type.
lengthRatioThresh = config['lengthRatioThresh']  # If length ratio < 3.0, set to dual_axis or else fixed_axis_diagonal, else single- or fixed-axis
areaRatioThresh = config['areaRatioThresh']  # If area ratio < 0.15, set to fixed_diag_axis, else dual_axis

# Set input CRS (native output from GEE) and a toCRS (EPSG:5070)
# geeCRS = 'EPSG:4326'
# toCRS = 'EPSG:5070'

# Set image resolution
res = 0.6

# Set dimensions
dim = 256 # 256x256 images

# Set the tile area proprtion: Proportional area of the array that sets the number of tiles
tileAreaProp = 0.5 # 50% of the array, so if an array is 1 km2, will generate tiles for 0.2 km2 within the array (or one for small arrays)

## Helper Functions

In [18]:
# Function to check if folder exists, if not create it
def checkFolder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

# Function to check for and remove erroneous geometries in arrays
def checkArrayGeometries(arrays): 
    # For a collection of reasons, array boundaries may contain erroneous geometries that result in a near-zero area, linestrings, or points. 
    # To check for and remove these, we'll explode arrays, calculate a temporary area, remove subarrays that are less than a minimum area, then dissolve by tempID.
    arrays['tempDissolveID'] = (1 + np.arange(len(arrays)))  # Create a temporary ID for dissolving
    arrays = arrays.explode(index_parts=False)
    arrays['tempArea'] = arrays['geometry'].area
    arrays = arrays[arrays['tempArea'] >= minPanelRowArea]
    arrays = arrays.dissolve(by=['tempDissolveID'], as_index=False)
    arrays = arrays.drop(columns=['tempArea', 'tempDissolveID'])
    arrays = arrays.reset_index(drop=True)
    return arrays

# Function to create an array from a set of panel rows based on the distance between them
def createArrayFromPanels(panels, buffDist, dissolveID, areaID='area'):
 
    # Count panels per group before dissolving
    panelCounts = panels.groupby(dissolveID).size().reset_index(name='numPanels')

    # Get the total area of the panels within each group (sum of area column). 
    panelAreas = panels.groupby(dissolveID)[areaID].sum().reset_index(name='pnlArea')
    
    # Buffer the geometries by buffDist, dissovle boundaries, and unbuffer by buffDist* -1. Assign the number of objects being dissovle into a numPanels column.
    arrays = panels.copy()
    arrays['geometry'] = arrays.buffer(buffDist)
    arrays = arrays.dissolve(by=[dissolveID], as_index=False)
    arrays['geometry'] = arrays.buffer(buffDist * -1)

    # Merge the panel counts and panel areas back into the dissolved array DataFrame. Select only the dissolveID and respective columns in the right df
    arrays = arrays.merge(panelCounts[[dissolveID, 'numPanels']], on=dissolveID, how='left')
    arrays = arrays.merge(panelAreas[[dissolveID, 'pnlArea']], on=dissolveID, how='left')

    # Due to the buffering and unbuffering, some mulitpolygons contain erroneous geometries that result in a near-zero area, linestrings, or points. Remove these.
    arrays = checkArrayGeometries(arrays)

    # Reset index
    arrays = arrays.reset_index(drop=True)
    return arrays

# Step 3: Reprocess and Mosaic Duplicate Files
def merge_rasters(folder_path, output_file):
    """Merges all raster files in a folder and keeps the CRS of the first image."""
    
    # Find all TIFF files in the folder
    tiff_files = glob.glob(os.path.join(folder_path, '*.tif'))
    
    if not tiff_files:
        print(f"No TIFF files found in {folder_path}. Skipping.")
        return
    
    # Open and read TIFF files
    src_files_to_mosaic = [rasterio.open(f) for f in tiff_files]
    
    # Merge files
    mosaic, out_trans = merge(src_files_to_mosaic)
    
    # Get metadata of the first file
    out_meta = src_files_to_mosaic[0].meta.copy()
    
    # Update metadata with new dimensions and transform
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "crs": src_files_to_mosaic[0].crs  # Keep CRS from first file
    })

    # Save mosaic
    with rasterio.open(output_file, "w", **out_meta) as dest:
        for i in range(1, mosaic.shape[0] + 1):
            dest.write(mosaic[i - 1], i)

    # Close files
    for src in src_files_to_mosaic:
        src.close()

    print(f"Mosaic created: {output_file}")

# Process Imagery into Images and Masks

## Setup

In [19]:
# Check if folders exist
checkFolder(imagesPath)
checkFolder(masksPath)

# Call gmseus arrays and panels
gmseusArrays = gpd.read_file(gmseusArraysFinalPath)
gmseusPanels = gpd.read_file(gmseusPanelsFinalPath)

# Select panel-rows where Source is "CCVPV" or "gmseus"
gmseusPanels = gmseusPanels[(gmseusPanels['Source'] == 'CCVPV') | (gmseusPanels['Source'] == 'gmseus')]

# Create arrays from panels
gmseusPanelsArrays = createArrayFromPanels(gmseusPanels, panelArrayBuff, 'arrayID', 'rowArea')

# Limit gmseusPanelsArrays to arrays with at least 10 numPanels
gmseusPanelsArrays = gmseusPanelsArrays[gmseusPanelsArrays['numPanels'] >= 10]

# Calculate a totArea column for each array
gmseusPanelsArrays['totArea'] = gmseusPanelsArrays['geometry'].area

# Set number of tiles to generate per array
tileArea = dim * dim * res * res  # Area of each tile in square meters

# Set the number of tiles to be (totArea / tileArea) * tileAreaProp OR 1 if the result is less than 1
gmseusPanelsArrays['numTiles'] = gmseusPanelsArrays['totArea'].apply(lambda x: max(1, int((x / tileArea) * tileAreaProp)))

# Print the sum of the number of tiles
print("Total number of image & mask pairs to generate: ", gmseusPanelsArrays['numTiles'].sum())

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ## Check for Tiled GEE Exports and Mosaic

# Due to export file size limitations, GEE can tile images on export resulting in multiple .tif files for a single arrayID. 
# We do not want to duplicate the number of tiles for this array for an inappropriate amount of area only because there are multiple files. 
# Thus, we mosaic them here, saving the originals. 

# Get filenames in the labelImgsWholePath
labelImgsWholeFiles = os.listdir(labelImgsWholePath)

# Extract arrayID from each filename. array_id is the integer value between 'id' and '.tif', but can also look like this id14432-0000000000-0000000000.tif where the arrayID is 14432
arrayIDs = []
file_map = {}  # Store filenames by arrayID
for f in labelImgsWholeFiles:
    match = re.search(r'id(\d+)', f)
    if match:
        array_id = int(match.group(1))
        arrayIDs.append(array_id)
        file_map.setdefault(array_id, []).append(f)

# Find duplicates
arrayID_counts = Counter(arrayIDs)
duplicateIDs = [arrayID for arrayID, count in arrayID_counts.items() if count > 1]

# # Print the number of duplicates
# print("Number of duplicate array whole tiffs (from GEE export tiling): ", len(duplicateIDs))
# # # Print the number of labelImgsWholeFiles that contain 0000000000
# # print("Number of labelImgsWholeFiles that contain 0000000000 (tiled images): ", len([f for f in labelImgsWholeFiles if '0000000000' in f]))
# # # Print all filenames that contain 0000000000
# # print([f for f in labelImgsWholeFiles if '0000000000' in f])

# ~~~~~~~~~~~~~~~~~~~~~~~~ Considered mosaicing duplicate, too memory intensive

# # Check if folders exist
# checkFolder(duplicateOrigPath)
# Move duplicate files to duplicateOrigPath folder to save them
# for array_id in duplicateIDs:
#     for filename in file_map[array_id]:
#         src_path = os.path.join(labelImgsWholePath, filename)
#         dest_path = os.path.join(duplicateOrigPath, filename)
#         shutil.move(src_path, dest_path)  # Move file
#         print(f"Moved {filename} to {duplicateOrigPath}")
# # Merge duplicate files
# for array_id in duplicateIDs:
#     folder_path = duplicateOrigPath
#     output_file = os.path.join(labelImgsWholePath, f"id{array_id}_mosaic.tif")
#     # Merge and save
#     merge_rasters(folder_path, output_file)

# ~~~~~~~~~~~~~~~~~~~~~~~~ We now simply adjust numTiles by image tile area within following loop

Total number of image & mask pairs to generate:  17586


## Split & Tile Whole Imagery

In [20]:
# This cell takes ~150 minutes to run 

# Loop through each labeled image and extract tiles 
for image_filename in os.listdir(labelImgsWholePath):
    if not image_filename.endswith('.tif'):
        continue  # Skip non-TIFF files

    # Extract arrayID from filename. array_id is the integer value between 'id' and '.tif', but can also look like this id14432-0000000000-0000000000.tif where the arrayID is 14432
    match = re.search(r'id(\d+)', image_filename)  # Looks for 'id' followed by digits
    if match:
        array_id = int(match.group(1))  # Convert to integer
    else:
        print(f"Warning: Could not extract array ID from {image_filename}. Skipping.")
        continue  # Skip this file if no match found

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Call in and prepare the image

    # Open the geotiff image
    image_path = os.path.join(labelImgsWholePath, image_filename)
    with rasterio.open(image_path) as src:
        print(f"Processing {image_filename} - Original CRS: {src.crs}")

        # Use the image's existing CRS (don't force-set it). If it's missing, print a warning and pass this image
        src_crs = src.crs
        if not src_crs:
            print(f"Warning: No CRS found for image {image_filename}. Skipping.")
            continue

        # Set toCRS to the image's CRS
        toCRS = src_crs

        # Set reprojected_src to the original src
        reprojected_src = src

        # # Check if reprojection is needed -- For now, maintain the original CRS
        # if src_crs != toCRS:
        #     print(f"Reprojecting {image_filename} to {toCRS}...")

        #     # Calculate new transformation
        #     transform, width, height = calculate_default_transform(
        #         src_crs, toCRS, src.width, src.height, *src.bounds
        #     )

        #     # Update metadata for reprojected image
        #     kwargs = src.meta.copy()
        #     kwargs.update({
        #         'crs': toCRS,
        #         'transform': transform,
        #         'width': width,
        #         'height': height
        #     })

        #     # Perform reprojection and keep MemoryFile open
        #     memfile = rasterio.MemoryFile()
        #     with memfile.open(**kwargs) as dst:
        #         for i in range(1, src.count + 1):
        #             reproject(
        #                 source=rasterio.band(src, i),
        #                 destination=rasterio.band(dst, i),
        #                 src_transform=src.transform,
        #                 src_crs=src_crs,
        #                 dst_transform=transform,
        #                 dst_crs=toCRS,
        #                 resampling=Resampling.nearest
        #             )

        #     # Keep the MemoryFile dataset open for reading
        #     reprojected_src = memfile.open()  # Keep file open!
        # else:
        #     reprojected_src = src  # Use original if no reprojection needed

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Get array geometry, number of tiles, and image bounds

        # Find the corresponding polygon for this array ID -- from gmseusPanelsArrays, to ensure panel-rows exist in image region
        array_row = gmseusPanelsArrays[gmseusPanelsArrays['arrayID'] == array_id]
        if array_row.empty:
            print(f"Warning: No matching polygon found for array ID {array_id}. Skipping.")
            continue

        # Trasform array_row to desired CRS and get the geometry
        array_row = array_row.to_crs(toCRS)
        array_geom = array_row.geometry.values[0]

        # Get the number of tiles to generate for this array
        numTilesArray = array_row['numTiles'].values[0]

        # Get raster bounds as a polygon
        image_bounds = reprojected_src.bounds
        image_poly = box(image_bounds.left, image_bounds.bottom, image_bounds.right, image_bounds.top)

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Check for duplicate images and area, adjust numTiles accordingly

        # Adjust numTiles based on proportional area if this array has duplicates
        if array_id in duplicateIDs:
            # Get all files for this duplicate array_id
            duplicate_files = [f for f in os.listdir(labelImgsWholePath) if re.search(r'id(\d+)', f) and int(re.search(r'id(\d+)', f).group(1)) == array_id]

            # Calculate current image area
            image_area = image_poly.area  # Area of the image portion that contains valid panels

            # Calculate total area across all duplicate images
            total_image_area = sum(box(*rasterio.open(os.path.join(labelImgsWholePath, f)).bounds).area for f in duplicate_files)

            # Adjust `numTiles` proportionally
            if total_image_area > 0:
                numTiles = max(1, int((image_area / total_image_area) * numTilesArray))  # Ensure at least 1 tile
            else:
                numTiles = 1  # Fallback to avoid division by zero

            # Print original numTiles and adjusted numTiles
            print(f"Original numTiles for {image_filename}: {numTilesArray}")    
            print(f"Adjusted numTiles for {image_filename}: {numTiles} based on proportional area.")
        else:
            numTiles = numTilesArray  # If no duplicate, keep original value

        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Generate random points within the array polygon and create tiles

        # Check if array polygon intersects the image
        if not array_geom.intersects(image_poly):
            print(f"Warning: Array {array_id} does not overlap with image {image_filename}. Skipping.")
            continue  # Skip this image

        # Generate numTiles number of random points inside the polygon
        for tile_idx in range(numTiles):
            while True:
                # Generate a random point within the bounds of the polygon
                minx, miny, maxx, maxy = array_geom.bounds
                random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
                
                # Ensure point is inside the polygon
                if array_geom.contains(random_point):
                    break
            
            # Convert point to pixel coordinates using the correct DatasetReader
            x, y = random_point.x, random_point.y
            row, col = reprojected_src.index(x, y) 

            # Define window for cropping the raster (ensuring it fits inside the raster bounds)
            window = Window(col - dim // 2, row - dim // 2, dim, dim)

            # Read bands
            img_bands = reprojected_src.read([1, 2, 3, 4], window=window)  # Assuming bands are R, G, B, N
            mask_band = reprojected_src.read(5, window=window)  # Assuming mask is stored in band 5

            # Save image bands
            img_outfile = os.path.join(imagesPath, f"id{array_id}_tile{tile_idx}.tif")
            with rasterio.open(
                img_outfile,
                'w',
                driver='GTiff',
                height=dim,
                width=dim,
                count=4,
                dtype=img_bands.dtype,
                crs=toCRS,
                transform=reprojected_src.window_transform(window)
            ) as dst:
                for i in range(4):
                    dst.write(img_bands[i], i + 1)

            # Save mask band
            mask_outfile = os.path.join(masksPath, f"id{array_id}_tile{tile_idx}.tif")
            with rasterio.open(
                mask_outfile,
                'w',
                driver='GTiff',
                height=dim,
                width=dim,
                count=1,
                dtype=mask_band.dtype,
                crs=toCRS,
                transform=reprojected_src.window_transform(window)
            ) as dst:
                dst.write(mask_band, 1)

            # Print message -- check
            # print(f"Saved image: {img_outfile} and mask: {mask_outfile}")

        # Close reprojected dataset after processing all tiles
        reprojected_src.close()
        if 'memfile' in locals():  # Close MemoryFile if it was used
            memfile.close()


Processing id100.tif - Original CRS: EPSG:26910
Processing id1001.tif - Original CRS: EPSG:26918
Processing id10014.tif - Original CRS: EPSG:26918
Processing id10029.tif - Original CRS: EPSG:26910
Processing id1003.tif - Original CRS: EPSG:26918
Processing id10036.tif - Original CRS: EPSG:26919
Processing id10038.tif - Original CRS: EPSG:26917
Processing id1004.tif - Original CRS: EPSG:26918
Processing id10045.tif - Original CRS: EPSG:26916
Processing id10056.tif - Original CRS: EPSG:26918
Processing id10058.tif - Original CRS: EPSG:26917
Processing id10059.tif - Original CRS: EPSG:26918
Processing id1006.tif - Original CRS: EPSG:26918
Processing id10063.tif - Original CRS: EPSG:26917
Processing id10065.tif - Original CRS: EPSG:26918
Processing id10067.tif - Original CRS: EPSG:26917
Processing id10070.tif - Original CRS: EPSG:26919
Processing id10073.tif - Original CRS: EPSG:26918
Processing id10078.tif - Original CRS: EPSG:26918
Processing id10079.tif - Original CRS: EPSG:26914
Proces